In [53]:
# from timeit import default_timer as timer
# from datetime import timedelta

# start = timer()
# end = timer()
# print(timedelta(seconds=end-start))


UsageError: Line magic function `%install_ext` not found.


In [54]:
def handle_response(url):
    response = requests.get(url)
    html = response.content

    soup = bs4.BeautifulSoup(html, features="html.parser")



    div = soup.find(class_ = 'title_wrapper')
    title = div.findChildren("h1" , recursive=False)[0]
    title = str(list(title.strings)[0]).strip()

    # title = str(list(soup.find(class_ = 'originalTitle').strings)[0]).strip()

    storyline = str(soup.find(class_ = 'inline canwrap').find("span").string).strip()

    f = soup.find(class_ = 'subtext')
    f = f.findAll("a")
    genres = []
    for a in f:
        genres.append(a.string)

    genres = genres[:len(genres) - 1]
    
    return {
        "title": title,
        "genre": "".join([" " + str(x) for x in genres]).strip(),
        "storyline": storyline
    }

In [55]:

import requests
import bs4

listOfGenres = ["Action", "Drama"]
movies_per_genre = 100

range_num = int(movies_per_genre / 50)
starts = [1 + 50 * i for i in range(range_num)]

title_ids = []

for genre in listOfGenres:
    baseUrl = ("https://www.imdb.com/search/title?title_type=feature&num_votes=25000,&genres=" + genre).strip()
    for start in starts:
        listUrl = (baseUrl + "&view=simple&sort=user_rating,desc&start=" + str(start) + "&ref_=adv_prv").strip()
        #print(listUrl)
        movieIDs = []
        movies = requests.get(listUrl)
        movieshtml = movies.content
        moviesSoup = bs4.BeautifulSoup(movieshtml, features="html.parser")
                
        for f in moviesSoup.find_all(class_ = 'lister-item mode-simple'):
            a = f.find('a', href=True)
            title_ids.append(a['href'])
        
        #sad treba uzeti svaki film iz liste, slično kao title
        #potencijalno odavdje izvući sve, ne ići na posebne stranice

In [56]:
title_ids[0]

'/title/tt5813916/'

In [57]:
len(title_ids)

200

In [58]:
base_url = 'https://www.imdb.com'
urls = list(map(lambda title_id: base_url + title_id, title_ids))

In [59]:
data_dict = []

for url in urls:
    movie = handle_response(url)
    data_dict.append(movie)

In [61]:
!pip install pandas

In [62]:
import pandas as pd

df = pd.DataFrame(data_dict)
df.head()

,genre,storyline,title
0,Action Drama War,DAG II; tells the story of a clandestine speci...,Dag II
1,Action Crime Drama,Set within a year after the events of Batman B...,Vitez tame
2,Action Adventure Sci-Fi,"Dom Cobb is a skilled thief, the absolute best...",Početak
3,Action Adventure Sci-Fi,After the devastating events of Avengers: Infi...,Osvetnici: Završnica
4,Action Sci-Fi,Thomas A. Anderson is a man living two lives. ...,Matrix
